In [1]:
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from tqdm import tqdm


c:\Users\PC\Documents\GitHub\ComputerVision-and-Deep-learning\NewRecipe\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# 모델과 토크나이저 로드
print("모델과 토크나이저를 로딩 중...")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

# GPU 사용 가능 시 GPU 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"사용 중인 디바이스: {device}")


모델과 토크나이저를 로딩 중...


c:\Users\PC\Documents\GitHub\ComputerVision-and-Deep-learning\NewRecipe\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PC\.cache\huggingface\hub\models--facebook--nllb-200-distilled-600M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


사용 중인 디바이스: cpu


In [1]:
print("error")

error


In [ ]:

# 한국어 언어 코드 ID 얻기
korean_lang_id = tokenizer.convert_tokens_to_ids("kor_Hang")

def translate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=korean_lang_id,
        max_length=100
    )
    return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

# CSV 파일 읽기
input_file = "/content/T1.csv"  # 입력 파일 이름을 적절히 변경하세요
print(f"{input_file} 파일을 읽는 중...")
df = pd.read_csv(input_file)
print(f"총 {len(df)} 행의 데이터를 읽었습니다.")

# 각 행 번역
print("번역을 시작합니다...")
translated_rows = []
for index, row in tqdm(df.iterrows(), total=len(df), desc="번역 진행"):
    translated_row = [translate_text(str(cell)) for cell in row]
    translated_rows.append(translated_row)
    print(f"행 {index+1}/{len(df)} 번역 완료")

# 번역된 데이터로 새로운 DataFrame 생성
translated_df = pd.DataFrame(translated_rows, columns=df.columns)

# 번역된 CSV 파일 저장
output_file = "/content/F1.csv"  # 출력 파일 이름을 적절히 변경하세요
print(f"번역된 데이터를 {output_file}에 저장 중...")
translated_df.to_csv(output_file, index=False)

print(f"번역이 완료되었습니다. 결과가 {output_file}에 저장되었습니다.")